<center><h1>Feature Reduction & Starter Models</h1></center>

In this notebook, we'll try different techniques to reduce the number of dimensions of our data.<br><br>
I'll try the following techniques:
<br>

**1. Feature Selection:**
- Forward Feature Selection.
- Backward Feature Selection.
- Remove High Collinarity Features.
- Recursive Feature Elimination method.
- Nearest Neighbors for Feature Extraction.

**2. Dimensionality Reduction:**
- PCA.
- ICA.
- t-SNE.
- UMAP.
- Random Projection.
- other.

For the starter models, I'll try the following models:<br>
1. Linear Regression.
1. Lasso.
2. Ridge.
3. Elastic Net.
4. SVR.
5. GaussianNB.
6. KNeighborsRegressor.
7. MLP.
8. RandomForestRegressor.
9. ExtraTreeRegressor.
10. GTBM (Sklearn)
11. XGBoost (Stanford).
12. LightGBM (Microsoft).
13. Catboost (Yandex).
14. Average Stacking.
15. Meta layer Stacking.

# Data Overview:
## Import Libraries:

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from tqdm import tqdm_notebook as tqdm

from util import *

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Read Data:

In [23]:
train = pd.read_csv('../3_Feature Engineering/output/train_engineered.csv').dropna()
test  = pd.read_csv('../3_Feature Engineering/output/test_engineered.csv')

train_labels = train.SalePrice

shape(train, test)

~> [train] has  1,408  rows, and  1,050  columns.
~> [test ] has  1,459  rows, and  1,049  columns.


In [50]:
ord_cols  = ['LandSlope', 'OverallQual', 'OverallCond', 'ExterQual', 'ExterCond', 'BsmtExposure', 
             'BsmtFinType1', 'BsmtCond', 'BsmtQual', 'BsmtFinType2', 'HeatingQC', 'Electrical', 
             'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'KitchenQual', 'TotRmsAbvGrd', 'Functional',
             'Fireplaces', 'FireplaceQu', 'GarageFinish', 'GarageCars', 'GarageQual', 'GarageCond', 'PavedDrive',
             'PoolQC', 'BedroomAbvGr', 'KitchenAbvGr']
cat_cols  = ['MSSubClass', 'MSZoning', 'Alley', 'LotShape', 'LandContour', 'LotConfig', 
            'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'RoofStyle', 'RoofMatl', 'Exterior1st',
            'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'GarageType', 'MiscFeature', 'SaleType', 
             'SaleCondition', 'HouseStyle', 'Fence', 'CentralAir']
num_cols  = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
            '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
            'EnclosedPorch', '3SsnPorch', 'ScreenPorch']
date_cols = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'MoSold', 'YrSold']

with open('../3_Feature Engineering/freq_cols.pkl', 'rb') as f:
    fre_cols = pickle.load(f)

with open('../3_Feature Engineering/log_cols.pkl', 'rb') as f:
    log_cols = pickle.load(f)
    
with open('../3_Feature Engineering/onehot_cols.pkl', 'rb') as f:
    onehot_cols = pickle.load(f)
    
with open('../3_Feature Engineering/poly_cols.pkl', 'rb') as f:
    poly_cols = pickle.load(f)
    
with open('../3_Feature Engineering/sqrt_cols.pkl', 'rb') as f:
    sqrt_cols = pickle.load(f)

with open('../3_Feature Engineering/target_enc_cols.pkl', 'rb') as f:
    target_enc_cols = pickle.load(f)
    
thermo_cols = [col for col in train.columns if col.startswith('thermo_')]

In [51]:
                    len(set(thermo_cols+
                           sqrt_cols+
                           target_enc_cols+
                           poly_cols+
                           onehot_cols+
                           log_cols+
                           fre_cols+
                           date_cols+
                           num_cols+
                           ord_cols+
                           cat_cols+list(train.columns)))

1050

In [66]:
sqrt_cols+thermo_cols+log_cols

[]

# Feature Selection:
## Remove High Correlated Features:

In [4]:
threshold = .9

# Create correlation matrix
corr = train.corr().abs()

# Select upper triangle of correlation matrix
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool))

# Select columns with correlations above threshold
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

print(f'~> There are {bg(len(to_drop))} columns to remove.  (;´༎ຶД༎ຶ`)')

~> There are  350  columns to remove.  (;´༎ຶД༎ຶ`)


In [5]:
train[to_drop].head()

,GarageQual,GarageCond,onehot_LandSlope_3,onehot_ExterQual_4,onehot_HeatingQC_5,onehot_Electrical_5,onehot_FullBath_1,onehot_FullBath_2,onehot_HalfBath_1,onehot_PavedDrive_3,...,diff_GarageYrBlt_MoSold,diff_GarageYrBlt_YrSold,diff_MoSold_YearBuilt,diff_MoSold_YearRemodAdd,diff_MoSold_GarageYrBlt,diff_MoSold_YrSold,diff_YrSold_YearBuilt,diff_YrSold_YearRemodAdd,diff_YrSold_GarageYrBlt,diff_YrSold_MoSold
0,3,3,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,2001.0,5.0,2001,2001,2001.0,2006,5,5,5.0,2006
1,3,3,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1971.0,31.0,1971,1971,1971.0,2002,31,31,31.0,2002
2,3,3,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1992.0,7.0,1992,1993,1992.0,1999,7,6,7.0,1999
3,3,3,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,...,1996.0,8.0,1913,1968,1996.0,2004,91,36,8.0,2004
4,3,3,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1988.0,8.0,1988,1988,1988.0,1996,8,8,8.0,1996


IMHO, maybe if we standardize/normalize these features, the result will differ.<br><br>
The thing I know, I won't depend on these result thoroughly.

## Recursive Feature Elimination Method:
For this notebook, I'll use RandomForest, to get some intuition about the features, but in the modeling section, I'll apply the same method for each model, and see how the result can differ.

In [28]:
# First, Define our metric target
from sklearn.metrics import mean_squared_error, make_scorer


def rmse(X_test, y_pred):
    return np.sqrt(mean_squared_error(X_test, y_pred))

scorer = make_scorer(rmse, greater_is_better=False)

In [29]:
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestRegressor as RFR

# train.drop('SalePrice', axis=1, inplace=True)

# Create a model for feature selection
estimator = RFR(random_state=10, n_estimators=100, n_jobs=-1)

# Create the object
selector = RFECV(estimator, step=1, cv=5, scoring=scorer, n_jobs=-1)

In [32]:
selector.fit(train.dropna(), train_labels)

KeyboardInterrupt: 

In [33]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score as acc
from mlxtend.feature_selection import SequentialFeatureSelector as sfs


# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    train,
    train_labels,
    test_size=0.2,
    random_state=42)

y_train = y_train.ravel()
y_test = y_test.ravel()

print('Training dataset shape:', X_train.shape, y_train.shape)
print('Testing dataset shape:', X_test.shape, y_test.shape)

Training dataset shape: (1126, 1050) (1126,)
Testing dataset shape: (282, 1050) (282,)


In [37]:
# Build RF classifier to use in feature selection
clf = RFR(n_estimators=100, n_jobs=-1)

# Build step forward feature selection
sfs1 = sfs(clf,
           k_features=1000,
           forward=False,
           floating=False,
           verbose=2,
           scoring=scorer,
           cv=2)

# Perform SFFS
sfs1 = sfs1.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.4s remaining:    0.0s

STOPPING EARLY DUE TO KEYBOARD INTERRUPT...